<a href="https://colab.research.google.com/github/DaHaaT/cifar-10/blob/main/cifar_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 必要なライブラリのインポート
import torch
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
# GPUが使えるかの確認
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

### CIFAR-10のデータセットをダウンロードする準備

In [ ]:
# 画像データの変換&データ拡張
# 訓練用・検証用
trainset_transform = transforms.Compose([
    transforms.ToTensor(), # 画像データをTensor型に変換 & [0,255] → [0,1]の正規化
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # RGB値の平均と標準偏差の値を予め設定して標準化 今回は[0,1] → [-1,1]のように変換している。
    transforms.RandomHorizontalFlip(p=0.4), # データ拡張
    transforms.RandomVerticalFlip(p=0.4), #データ拡張
    transforms.RandomRotation(degrees=[-45, 45]) # データ拡張
])

"""
CIFAR-10の元々の画像データはPIL形式
Tensor型への変換では, (縦, 横, 色チャネル数)を(色チャネル数, 縦, 横)に変換している
※ PIL形式のサイズは (縦, 横)
"""
# テスト用 (テスト用にはデータ拡張を行わない)
testset_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

### CIFAR-10のDatasetをダウンロード

In [ ]:
# 訓練用と検証用データの準備
train_validation_dataset = CIFAR10(root='./data', train=True, download=True, transform=trainset_transform)

# テスト用データの準備
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=testset_transform)

#CIFAR-10のデータセットに含まれる対象クラス (10クラス)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Datasetの数の確認

In [ ]:
print('訓練用データの数 + 検証用データの数: ', len(train_validation_dataset))
print('テスト用データの数: ', len(test_dataset))

### train_validation_datasetを訓練用と検証用に分割

In [ ]:
# train_validation_datasetを訓練用と検証用に分割 (少し時間が掛かる　約30秒～1分過ぎぐらい)
train_dataset, validation_dataset = train_test_split(train_validation_dataset, test_size=0.2, shuffle=True)
print('訓練用データの数: ', len(train_dataset))
print('検証用データの数: ', len(validation_dataset))
print('テスト用データの数: ', len(test_dataset))

### Dataloaderの作成 (ミニバッチ学習の準備)

In [ ]:
# バッチサイズの設定 ※ 1イテレーションごとに流す画像データの数
batch_size = 32

# Dataloaderの作成
# 画像データをミニバッチ(データの小さな集まり)に分けて学習させる準備。今回はデータを32個の塊としている (batch_size = 32)。
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
validationloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

"""
shuffle=Trueにすることでエポックが回るごとにミニバッチの中身がランダムに入れ替わる。
num_workersはミニバッチを作成する際の並列実行数
"""

# 辞書型にDataloaderを格納
dataloader_dict = {'train': trainloader, 'validation': validationloader}

### 1つのミニバッチの中身を確認

In [ ]:
# 訓練用のDataloaderからミニバッチを1つ抽出する
dataiter = iter(trainloader)
images, labels = next(dataiter)

In [ ]:
# 1つのミニバッチから画像情報を表示する
print('[画像データの数, 色チャネル数(RGB), 縦, 横]: ', images.shape)

In [ ]:
# 1つのミニバッチから正解ラベルを表示する
print('32個の正解ラベルを表示: ', labels)

### 画像データと正解ラベルの確認

In [ ]:
# 画像と正解ラベルを表示
def imshow(img):
    img = img / 2 + 0.5 # 最小値を0, 最大値を1にする
    img = img.numpy() # tensor配列をndarray配列に変換する
    img = np.transpose(img, (1, 2, 0)) # (色チャネル数, 縦, 横) → (縦, 横, 色チャネル数)に変換する
    plt.imshow(img)

# 5つの画像をランダムに表示
fig = plt.figure(figsize=(12, 5))
for i in range(5):
    n = np.random.choice(len(images))
    ax = fig.add_subplot(1, 5, i+1)
    imshow(images[n])
    ax.set_title(classes[labels[n]])
plt.show()

### CNNのモデルを定義

In [ ]:
# CNNのモデル構造を定義する
class CNN(nn.Module):
    def __init__(self, class_num):
        super(CNN, self).__init__()
        # 畳み込み層
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, padding=2) # 3チャネル(RGB)の画像データに対して64個のカーネルを用意する → 64個の特徴マップ(64個のチャネルを持つ画像データ)が出力される。
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1) # 64個のチャネルを持つ画像データに対して128個のカーネルを用意する → 128個の特徴マップ(128個のチャネルを持つ画像データ)が出力される。
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1) # 上と同じ流れ
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1) # 上と同じ流れ

        # 全結合層
        conv4_output_size = 2 * 2 * 512 # 全結合層に入る前：2x2の特徴マップが512個出力される
        self.fc1 = nn.Linear(in_features=conv4_output_size, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=class_num) # 最後の出力層のノードの数はクラス数と同じ10個

        # 活性化関数
        self.relu = nn.ReLU(inplace=True)
        self.softmax = nn.Softmax(dim=1)

        # プーリング層
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))
        x = torch.flatten(x, 1)  # 次元を1次元に落とす
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x) # 10個の確率を出力する
        return x

"""
classとは言わば設計図のようなもので, 設計図を定義した後にそこから実体(インスタンス)を作成する。
イメージ: 動物という設計図を定義したら, イヌ, ネコ, 人間という実体を作成することができる。
"""

model = CNN(class_num=len(classes)) # CNNという設計図を作成して, modelという実体(インスタンス)を作成した。
model.to(device) # modelをGPU上へ

### 学習に使う損失関数と最適化アルゴリズムを定義

In [ ]:
# 損失関数の定義
criterion = nn.CrossEntropyLoss() # 交差エントロピー誤差 (多クラス分類によく用いられる)

"""
2クラス分類 → バイナリ交差エントロピー誤差
多クラス分類 → 交差エントロピー誤差
"""

# 最適化アルゴリズム(勾配降下法)の定義
optimizer = optim.Adam(params=model.parameters(), lr=0.0001) # Adamというアルゴリズムを用いて勾配降下法を行う

### 学習の開始

In [ ]:
epoch_num = 20 # エポックの数 ※ 1epoch = 全データセットを1回使うこと, 10epoch = 10回全データセットを使ったことに相当する
train_losses = []
train_accs = []
validation_losses = []
validation_accs = []

# 学習 (Epoch=20で約3分掛かる)
for epoch in range(epoch_num):
    print('Epoch: {}/{}'.format(epoch+1, epoch_num))
    for phase in ['train', 'validation']:
        if phase == 'train':
            model.train() # モデルを訓練モードに設定
        else:
            model.eval() # モデルを検証モードに設定

        running_loss = 0.0
        running_acc = 0.0

        for imgs, labels in dataloader_dict[phase]:
            imgs = imgs.to(device) # GPU上へ
            labels = labels.to(device) # GPU上へ
            one_hot_labels = F.one_hot(labels, num_classes=len(classes)).float() # One-hot vectorに変換 例) ラベル7 → (0, 0, 0, 0, 0, 0, 1, 0, 0, 0)
            optimizer.zero_grad() # 重みパラメータの勾配を0に初期化
            outputs = model(imgs) # モデルの学習結果を出力する
            loss = criterion(outputs, one_hot_labels) # 正解値と予測値の損失差を計算する

            # 訓練時だけに適用
            if phase == 'train':
                loss.backward() # 誤差逆伝播を行い, 重みパラメータの勾配を求める
                optimizer.step() # 最適化アルゴリズムを用いて重みパラメータを更新する

            running_loss += loss.item() # 1イテレーションごとに損失差の値が追加されていく
            predicted_labels = torch.argmax(outputs, dim=1) # 一番大きい値を示したインデックス番号を抽出する
            running_acc += torch.mean(predicted_labels.eq(labels).float()) # 1イテレーションごとの正解率が追加されていく

        running_loss /= len(dataloader_dict[phase]) # イテレーションの数で割ることで, 平均損失差を計算して, 1epoch分の損失差を出力
        running_acc /= len(dataloader_dict[phase]) # イテレーションの数で割ることで, 平均正解率を計算して, 1epoch分の正解率を出力
        running_acc = running_acc.item()

        if phase == 'train':
            train_losses.append(running_loss)
            train_accs.append(running_acc)
            print('train loss: {:.3f}, train acc: {:.3f}'.format(running_loss, running_acc))

        else:
            validation_losses.append(running_loss)
            validation_accs.append(running_acc)
            print('validation loss: {:.3f}, validation acc: {:.3f}'.format(running_loss, running_acc))
    print('------------------------')

print('学習が終了しました。')

### 学習中の損失差と正解率の推移

In [ ]:
# 損失差の推移をプロット
fig = plt.figure(figsize=(8, 5))
plt.plot(train_losses, label='train', linewidth=2, color='coral')
plt.plot(validation_losses, label='validation', linewidth=2, color='dodgerblue')
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
plt.title('Loss', fontsize=20)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(loc='upper right')

In [ ]:
# 正解率の推移をプロット
fig = plt.figure(figsize=(8, 5))
plt.plot(train_accs, label='train', linewidth=2, color='coral')
plt.plot(validation_accs, label='validation', linewidth=2, color='dodgerblue')
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
plt.title('Accuracy', fontsize=20)
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(loc='lower right')

### 学習後のモデルパラメータを保存

In [ ]:
# 学習したモデルパラメータの保存
SAVED_PATH = './model_epoch_{}.pth'.format(epoch_num)
if not os.path.isfile(SAVED_PATH):
    torch.save(model.state_dict(), SAVED_PATH)

### テスト用CNNモデルの定義

In [ ]:
test_model = CNN(class_num=len(classes))
test_model.load_state_dict(torch.load(SAVED_PATH)) # 保存された重みパラメータを使う
test_model.eval() # 推論モードに切り替える

### テスト用のDataloaderの作成

In [ ]:
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

### 推論の開始

In [ ]:
# テスト用のDataloaderからミニバッチを1つ抽出する
dataiter = iter(testloader)
test_images, test_labels = next(dataiter)

In [ ]:
# 推論
with torch.no_grad(): # 勾配計算は行わないように設定
    test_images.to(device)
    outputs = test_model(test_images)
    predicted_labels = torch.argmax(outputs, dim=1)

### 正解クラスと予測クラスの比較

In [ ]:
# ランダムな画像を5つ表示
fig = plt.figure(figsize=(12, 5))
for i in range(5):
    n = np.random.choice(len(test_images))
    ax = fig.add_subplot(1, 5, i+1)
    imshow(test_images[n])
    correct_class = classes[test_labels[n]]
    predicted_class = classes[predicted_labels[n]]
    if correct_class == predicted_class:
        ax.set_xlabel('GOOD')
        ax.xaxis.label.set_color('red')
    else:
        ax.set_xlabel('BAD')
    ax.set_title('c = {}, p = {}'.format(correct_class, predicted_class)) # 正解クラスと予測クラスを見比べる
plt.show()

### Heatmapで表示

In [ ]:
testloader_2 = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False, num_workers=2)
dataiter = iter(testloader_2)
test_images, test_labels = next(dataiter)

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 約1分掛かる
with torch.no_grad():
    test_images.to(device)
    outputs = test_model(test_images)
    predicted_labels = torch.argmax(outputs, dim=1)
    for test_label, predicted_label in zip(test_labels, predicted_labels):
            if test_label == predicted_label:
                correct_pred[classes[test_label]] += 1
            total_pred[classes[test_label]] += 1

In [ ]:
print('クラスごとの正解率')
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'{classname:5s} is {accuracy:.1f} %')

In [ ]:
# Heatmapの表示
cm = confusion_matrix(predicted_labels, test_labels)
cm = pd.DataFrame(data=cm, index=classes, columns=classes)
plt.figure(figsize=(8, 8))
sns.heatmap(cm, square=True, cbar=True, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Correct class', fontsize=12)
plt.ylabel('Predicted class', fontsize=12)